In [ ]:
import csv
import openai
import requests
import json
import time
import random

openai.api_key = ""

INPUT_CSV = "units.csv"
OUTPUT_CSV = "results.csv"
PROMPT_CSV = "prompts_responses.csv"  # CSV to save prompts and responses
import requests

CUSTOM_PATHS = {
    ("spring-boot", "958a0a45f164601d01cb706c19f22ed3e25eff56"):
        "spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure/MongoProperties/merge.java",
    ("spring-boot", "3444ebbc05b99a164474c14d6a6784f749514428"):
        "spring-boot/src/main/java/org/springframework/boot/context/SpringBootServletInitializer/merge.java",
    ("spring-boot", "074771ec125dd407af0282b92960e9e9e3377e84"):
        "spring-boot/src/main/java/org/springframework/boot/context/SpringBootServletInitializer/merge.java",
    ("spring-boot", "fdd3f12ee0f92ac18844c08bf71df39feebb6673"):
        "spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure/AutoConfigurationReportLoggingInitializer/merge.java",
    ("spring-boot", "2d4e68a9777601bfb8309c94d8b74bc21be80ad1"):
        "src/main/java/org/springframework/boot/context/embedded/TomcatEmbeddedServletContainerFactory/merge.java"
}

def fetch_code_from_github(project, commit, class_name):
    """
    Fetches the full merge.java code from GitHub.
    Tries custom paths first if commit is known to be problematic.
    """
    base_url = "https://raw.githubusercontent.com/spgroup/mergedataset/master"
    key = (project, commit)
    
    urls_to_try = []
    
    if key in CUSTOM_PATHS:
        # Use the custom path provided
        urls_to_try.append(f"{base_url}/{project}/{commit}/source/{CUSTOM_PATHS[key]}")
    
    # Also try default conversion: class_name -> path
    class_path = class_name.replace(".", "/")
    urls_to_try.append(f"{base_url}/{project}/{commit}/source/{class_path}/merge.java")
    urls_to_try.append(f"{base_url}/{project}/{commit}/source/{class_path}.java")
    
    for url in urls_to_try:
        print(f"Trying URL: {url}")
        r = requests.get(url)
        if r.status_code == 200:
            print(f"✅ Code found at: {url}")
            return r.text
    
    print(f"❌ Code not found for {project}, commit {commit}, class {class_name}")
    return "// Code not found"


def call_gpt(prompt, retries=3):
    """
    Calls GPT-5 for analysis, with retry/backoff in case of errors.
    """
    for attempt in range(retries):
        try:
            response = openai.chat.completions.create(
                model="gpt-5",
                messages=[{"role": "user", "content": prompt}]
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            wait_time = (2 ** attempt) + random.random()
            print(f"API error ({e}), retrying in {wait_time:.1f}s...")
            time.sleep(wait_time)
    return json.dumps({"conflict": None, "explanation": "Failed after retries"})

with open(INPUT_CSV, newline="", encoding="utf-8-sig") as infile, \
     open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as outfile, \
     open(PROMPT_CSV, "w", newline="", encoding="utf-8") as promptfile:

    reader = csv.DictReader(infile, delimiter=";", quotechar='"')
    fieldnames = reader.fieldnames + ["conflict", "explanation"]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames, delimiter=";", quotechar='"')
    writer.writeheader()

    # CSV to save prompts and responses
    prompt_writer = csv.DictWriter(promptfile, fieldnames=["ID", "prompt", "response"], delimiter=";", quotechar='"')
    prompt_writer.writeheader()

    for i, row in enumerate(reader, 1):
        project = row["project"]
        commit = row["merge commit"]
        class_name = row["className"]
        method_name = row["method"]

        code = fetch_code_from_github(project, commit, class_name)

        if code is None:
            print(f"[{i}] {project}:{commit} -> ❌ Code not found, skipping GPT call")
            row["conflict"] = None
            row["explanation"] = "Code not found, GPT analysis skipped"
            writer.writerow(row)
            continue  # Skip to next row

        prompt = f"""
        You are a software engineering expert. Your task is to determine if there is a semantic merge conflict
        in a scenario where two developers have independently modified a base program.

        Step 1: Analyze the changes.
        - Carefully read and understand the changes made by the LEFT developer.
        - Carefully read and understand the changes made by the RIGHT developer.
        - Summarize in your own words what each change does and its potential effect on the program behavior.

        Step 2: Evaluate semantic conflict.
        - Definition: Separate changes Left and Right to a base program B interfere when integrated changes do not preserve the altered behavior of the left or right, or the unchanged behavior of the base.
        - Separate changes L and R to a base program B interfere if there is a state element x such that B, L and R compute different values for x.
        - Separate changes L and R to a base program B interfere if there is a state element x such that L (or R) computes a differente value for x than B and Merge
        -Separate changes L and R to a base program B interfere if there is a state element x such that B, L and R compute the same value for x but Merge computes a differente value.
        - Consider the context of the entire method and class, not just the modified lines.
        - Consider the overall program behavior, not just syntactic changes.
        - Consider refactorings as non-conflicting.

        Scenario information:
        - Project: {project}
        - Merge commit: {commit}
        - Class: {class_name}
        - Method: {method_name}
        - LEFT developer changes:
        - Modified lines: {row['left modifications']}
        - Summary: {row['Summary of changes made by left']}
        - RIGHT developer changes:
        - Modified lines: {row['right modifications']}
        - Summary: {row['Summary of changes made by right']}
        - Location of interest (LOI): {row['LOI']}

        Original code context:
        ```java
        {code}
        ````

        Question:
        Is there a semantic conflict between these changes?
        Answer in JSON format:
        {{
        "conflict": true|false,
        "explanation": "brief explanation"
        }}
        """

        # Call GPT-5
        answer = call_gpt(prompt)

        # Try to convert the response to JSON
        try:
            parsed = json.loads(answer)
            row["conflict"] = parsed.get("conflict", None)
            row["explanation"] = parsed.get("explanation", "")
        except:
            row["conflict"] = None
            row["explanation"] = answer

        # Save in main CSV
        writer.writerow(row)

        # Save prompt and response in extra CSV
        prompt_writer.writerow({
            "ID": row["ID"],
            "prompt": prompt,
            "response": answer
        })

        print(f"[{i}] {project}:{commit} -> {row['conflict']}")

        time.sleep(1)  # avoid GitHub / GPT rate limits


Trying URL: https://raw.githubusercontent.com/spgroup/mergedataset/master/activiti/50d8e43eb5917c63abfbcdec1e68e510943f325a/source/org/activiti/engine/impl/persistence/entity/DeploymentEntityManager/merge.java
✅ Code found at: https://raw.githubusercontent.com/spgroup/mergedataset/master/activiti/50d8e43eb5917c63abfbcdec1e68e510943f325a/source/org/activiti/engine/impl/persistence/entity/DeploymentEntityManager/merge.java
[1] activiti:50d8e43eb5917c63abfbcdec1e68e510943f325a -> False
Trying URL: https://raw.githubusercontent.com/spgroup/mergedataset/master/activiti/bf46684ba62f5883673ea8fb0a14aecfe0aedea2/source/org/activiti/engine/impl/bpmn/behavior/UserTaskActivityBehavior/merge.java
✅ Code found at: https://raw.githubusercontent.com/spgroup/mergedataset/master/activiti/bf46684ba62f5883673ea8fb0a14aecfe0aedea2/source/org/activiti/engine/impl/bpmn/behavior/UserTaskActivityBehavior/merge.java
[2] activiti:bf46684ba62f5883673ea8fb0a14aecfe0aedea2 -> False
Trying URL: https://raw.githubus

In [ ]:
import csv
import openai
import requests
import json
import time
import random

openai.api_key = ""

INPUT_CSV = "units.csv"
OUTPUT_CSV = "results_without_summary.csv"
PROMPT_CSV = "prompts_responses_without_summary.csv"  # CSV to save prompts and responses
import requests

CUSTOM_PATHS = {
    ("spring-boot", "958a0a45f164601d01cb706c19f22ed3e25eff56"):
        "spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure/MongoProperties/merge.java",
    ("spring-boot", "3444ebbc05b99a164474c14d6a6784f749514428"):
        "spring-boot/src/main/java/org/springframework/boot/context/SpringBootServletInitializer/merge.java",
    ("spring-boot", "074771ec125dd407af0282b92960e9e9e3377e84"):
        "spring-boot/src/main/java/org/springframework/boot/context/SpringBootServletInitializer/merge.java",
    ("spring-boot", "fdd3f12ee0f92ac18844c08bf71df39feebb6673"):
        "spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure/AutoConfigurationReportLoggingInitializer/merge.java",
    ("spring-boot", "2d4e68a9777601bfb8309c94d8b74bc21be80ad1"):
        "src/main/java/org/springframework/boot/context/embedded/TomcatEmbeddedServletContainerFactory/merge.java"
}

def fetch_code_from_github(project, commit, class_name):
    """
    Fetches the full merge.java code from GitHub.
    Tries custom paths first if commit is known to be problematic.
    """
    base_url = "https://raw.githubusercontent.com/spgroup/mergedataset/master"
    key = (project, commit)
    
    urls_to_try = []
    
    if key in CUSTOM_PATHS:
        # Use the custom path provided
        urls_to_try.append(f"{base_url}/{project}/{commit}/source/{CUSTOM_PATHS[key]}")
    
    # Also try default conversion: class_name -> path
    class_path = class_name.replace(".", "/")
    urls_to_try.append(f"{base_url}/{project}/{commit}/source/{class_path}/merge.java")
    urls_to_try.append(f"{base_url}/{project}/{commit}/source/{class_path}.java")
    
    for url in urls_to_try:
        print(f"Trying URL: {url}")
        r = requests.get(url)
        if r.status_code == 200:
            print(f"✅ Code found at: {url}")
            return r.text
    
    print(f"❌ Code not found for {project}, commit {commit}, class {class_name}")
    return "// Code not found"



def call_gpt(prompt, retries=3):
    """
    Calls GPT-5 for analysis, with retry/backoff in case of errors.
    """
    for attempt in range(retries):
        try:
            response = openai.chat.completions.create(
                model="gpt-5",
                messages=[{"role": "user", "content": prompt}]
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            wait_time = (2 ** attempt) + random.random()
            print(f"API error ({e}), retrying in {wait_time:.1f}s...")
            time.sleep(wait_time)
    return json.dumps({"conflict": None, "explanation": "Failed after retries"})


with open(INPUT_CSV, newline="", encoding="utf-8-sig") as infile, \
     open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as outfile, \
     open(PROMPT_CSV, "w", newline="", encoding="utf-8") as promptfile:

    reader = csv.DictReader(infile, delimiter=";", quotechar='"')
    fieldnames = reader.fieldnames + ["conflict", "explanation"]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames, delimiter=";", quotechar='"')
    writer.writeheader()

    # CSV to save prompts and responses
    prompt_writer = csv.DictWriter(promptfile, fieldnames=["ID", "prompt", "response"], delimiter=";", quotechar='"')
    prompt_writer.writeheader()

    for i, row in enumerate(reader, 1):
        project = row["project"]
        commit = row["merge commit"]
        class_name = row["className"]
        method_name = row["method"]

        code = fetch_code_from_github(project, commit, class_name)

        if code is None:
            print(f"[{i}] {project}:{commit} -> ❌ Code not found, skipping GPT call")
            row["conflict"] = None
            row["explanation"] = "Code not found, GPT analysis skipped"
            writer.writerow(row)
            continue  # Skip to next row

        prompt = f"""
        You are a software engineering expert. Your task is to determine if there is a semantic merge conflict
        in a scenario where two developers have independently modified a base program.

        Step 1: Analyze the changes.
        - Carefully read and understand the changes made by the LEFT developer.
        - Carefully read and understand the changes made by the RIGHT developer.
        - Summarize in your own words what each change does and its potential effect on the program behavior.

        Step 2: Evaluate semantic conflict.
        - Definition: Separate changes Left and Right to a base program B interfere when integrated changes do not preserve the altered behavior of the left or right, or the unchanged behavior of the base.
        - Separate changes L and R to a base program B interfere if there is a state element x such that B, L and R compute different values for x.
        - Separate changes L and R to a base program B interfere if there is a state element x such that L (or R) computes a differente value for x than B and Merge
        - Separate changes L and R to a base program B interfere if there is a state element x such that B, L and R compute the same value for x but Merge computes a differente value.
        - Consider the context of the entire method and class, not just the modified lines.
        - Consider the overall program behavior, not just syntactic changes.
        - Consider refactorings as non-conflicting.


        Scenario information:
        - Project: {project}
        - Merge commit: {commit}
        - Class: {class_name}
        - Method: {method_name}
        - LEFT developer changes:
        - Modified lines: {row['left modifications']}
        - RIGHT developer changes:
        - Modified lines: {row['right modifications']}
        - Location of interest (LOI): {row['LOI']}

        Original code context:
        ```java
        {code}
        ````

        Question:
        Is there a semantic conflict between these changes?
        Answer in JSON format:
        {{
        "conflict": true|false,
        "explanation": "brief explanation"
        }}
        """

        # Call GPT-5
        answer = call_gpt(prompt)

        # Try to convert the response to JSON
        try:
            parsed = json.loads(answer)
            row["conflict"] = parsed.get("conflict", None)
            row["explanation"] = parsed.get("explanation", "")
        except:
            row["conflict"] = None
            row["explanation"] = answer

        # Save in main CSV
        writer.writerow(row)

        # Save prompt and response in extra CSV
        prompt_writer.writerow({
            "ID": row["ID"],
            "prompt": prompt,
            "response": answer
        })

        print(f"[{i}] {project}:{commit} -> {row['conflict']}")

        time.sleep(1)  # avoid GitHub / GPT rate limits


Trying URL: https://raw.githubusercontent.com/spgroup/mergedataset/master/activiti/50d8e43eb5917c63abfbcdec1e68e510943f325a/source/org/activiti/engine/impl/persistence/entity/DeploymentEntityManager/merge.java
✅ Code found at: https://raw.githubusercontent.com/spgroup/mergedataset/master/activiti/50d8e43eb5917c63abfbcdec1e68e510943f325a/source/org/activiti/engine/impl/persistence/entity/DeploymentEntityManager/merge.java
[1] activiti:50d8e43eb5917c63abfbcdec1e68e510943f325a -> False
Trying URL: https://raw.githubusercontent.com/spgroup/mergedataset/master/activiti/bf46684ba62f5883673ea8fb0a14aecfe0aedea2/source/org/activiti/engine/impl/bpmn/behavior/UserTaskActivityBehavior/merge.java
✅ Code found at: https://raw.githubusercontent.com/spgroup/mergedataset/master/activiti/bf46684ba62f5883673ea8fb0a14aecfe0aedea2/source/org/activiti/engine/impl/bpmn/behavior/UserTaskActivityBehavior/merge.java
[2] activiti:bf46684ba62f5883673ea8fb0a14aecfe0aedea2 -> False
Trying URL: https://raw.githubus